Notebook objective: Train & tune GPBoost Booster + random effect model. Goal is to compare the Booster's learning with 2.0LGBM, and determine if it is training properly.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import optuna

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import StratifiedKFold
import gpboost as gpb

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Set print options
np.set_printoptions(suppress=True, precision=6, edgeitems = 7)
pd.options.display.float_format = '{:.6f}'.format
pd.set_option('display.max_columns', None)

## Preprocessing

In [3]:
random_state = 1923

In [4]:
# Read data
df = pd.read_csv("./OutputData/training_data.csv")

In [5]:
# Split features, groupings and target
X = df.drop(["life_expectancy", "country", "year"], axis = 1)
G = df[["country", "year"]]
y = df.life_expectancy

In [6]:
# Create CV splitter, to be stratified by country
stratify = df["country"]
cv = StratifiedKFold(n_splits = 4)

## Hyperparameter tuning

In [7]:
# Objective function (Sklearn API)
def objective_gpb(trial):

    # Define hyperparameter space
    learning_rate = trial.suggest_float("learning_rate", 0.05, 0.5)
    num_leaves = trial.suggest_int("num_leaves", 2**2, 2**10)
    min_child_samples = trial.suggest_int("min_child_samples", 10, 1000, log = True)
    min_child_weight = trial.suggest_int("min_child_weight", 0.001, 20)
    reg_alpha = trial.suggest_float("l1_reg", 5e-5, 1, log = True)
    reg_lambda = trial.suggest_float("l2_reg", 0, 2)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.25, 1)

    # Store tuning scores & number of rounds
    scores = []
    rounds = []

    for i, (train_index, val_index) in enumerate(cv.split(X, stratify)):

        # Split train - val
        X_train, G_train, y_train = X.iloc[train_index, ], G.iloc[train_index, ], y[train_index]
        X_val, G_val, y_val = X.iloc[val_index, ], G.iloc[val_index, ], y[val_index]
    
        # Create model
        model = gpb.GPBoostRegressor(
        n_jobs = 10,
        #device_type = "gpu",
        n_estimators = 5000,
        num_leaves = num_leaves,
        random_state = random_state,
        max_depth = -1,
        min_child_samples = min_child_samples,
        learning_rate = learning_rate,
        min_child_weight = min_child_weight,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        colsample_bytree = colsample_bytree
        )

        # Create random effects model
        gp_model = gpb.GPModel(
            group_data = G_train, # Random intercepts for each group
            likelihood = "gaussian",
            seed = random_state
        )
        gp_model.set_prediction_data(group_data_pred = G_val)

        # Train model with early stopping
        model.fit(
            X_train, 
            y_train,
            gp_model = gp_model, 
            eval_set = [(X_val, y_val)],
            early_stopping_rounds = 50,
            verbose = False)

        # Record best number of rounds
        rounds.append(model.best_iteration_)

        # Record best score
        scores.append(model.best_score_['valid_0']['l2'])

    # Report mean number of rounds
    trial.set_user_attr("n_rounds", (np.mean(rounds)))
    
    return np.mean(scores)


In [8]:
# Create study
study_gpb = optuna.create_study(
  sampler = optuna.samplers.TPESampler(seed = random_state),
  study_name = "tune_gpb",
  direction = "minimize"
)

[I 2023-12-01 17:01:10,136] A new study created in memory with name: tune_gpb


In [10]:
# Perform study
optuna.logging.set_verbosity(optuna.logging.ERROR)
study_gpb.optimize(
  objective_gpb, 
  n_trials = 100,
  show_progress_bar = True)

Best trial: 65. Best value: 4.93979: 100%|███████████████████████████████████████████| 100/100 [03:26<00:00,  2.07s/it]


In [11]:
# Save tuning log
trials_gpb = study_gpb.trials_dataframe().sort_values("value", ascending = True)
trials_gpb.to_csv("./OutputData/trials_gpb.csv", index = False)

## Findings

In [33]:
# Load best 10 tunes
best_tunes = pd.read_csv("./OutputData/trials_gpb.csv")
best_tunes = best_tunes[best_tunes["state"] == "COMPLETE"]

In [34]:
best_tunes

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_l1_reg,params_l2_reg,params_learning_rate,params_min_child_samples,params_min_child_weight,params_num_leaves,params_subsample,user_attrs_n_rounds,state
0,65,4.939794,2023-12-01 17:03:58.693094,2023-12-01 17:04:01.261836,0 days 00:00:02.568742,0.530580,0.100626,1.775920,0.221216,97,5,547,0.769121,293.500000,COMPLETE
1,95,5.014928,2023-12-01 17:04:46.503644,2023-12-01 17:04:48.393917,0 days 00:00:01.890273,0.502079,0.012275,1.994347,0.229841,93,5,606,0.780057,251.500000,COMPLETE
2,90,5.020470,2023-12-01 17:04:39.585267,2023-12-01 17:04:41.174411,0 days 00:00:01.589144,0.510911,0.024867,1.891242,0.232190,113,5,465,0.795210,259.750000,COMPLETE
3,58,5.036119,2023-12-01 17:03:47.307570,2023-12-01 17:03:49.008804,0 days 00:00:01.701234,0.550464,0.062265,1.989084,0.152167,74,4,627,0.823939,216.000000,COMPLETE
4,79,5.052757,2023-12-01 17:04:23.362410,2023-12-01 17:04:24.895930,0 days 00:00:01.533520,0.521896,0.016445,1.698376,0.245469,99,6,485,0.862621,200.250000,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,5.976092,2023-12-01 17:01:29.941850,2023-12-01 17:01:34.092797,0 days 00:00:04.150947,0.735950,0.000236,0.448577,0.379245,440,5,900,0.668779,1154.750000,COMPLETE
96,2,6.024621,2023-12-01 17:01:34.094297,2023-12-01 17:01:36.943684,0 days 00:00:02.849387,0.282639,0.000504,1.351734,0.337727,488,8,934,0.863708,769.500000,COMPLETE
97,9,6.081029,2023-12-01 17:01:50.307257,2023-12-01 17:01:55.080277,0 days 00:00:04.773020,0.298754,0.055072,1.726511,0.063392,480,18,206,0.956369,1460.000000,COMPLETE
98,7,10.222605,2023-12-01 17:01:46.434415,2023-12-01 17:01:47.321995,0 days 00:00:00.887580,0.883639,0.001030,1.800125,0.318041,850,7,776,0.543593,18.500000,COMPLETE


Most tunes train for hundreds of rounds, and the hyperparameter configuration has a strong impact on model performance. This suggests there is no code issue with GPBoost.

In [23]:
# Test fixed effect predictions' range with a final model

for i, (train_index, val_index) in enumerate(cv.split(X, stratify)):

    if i > 0:
        break

    # Split train - val
    X_train, G_train, y_train = X.iloc[train_index, ], G.iloc[train_index, ], y[train_index]
    X_val, G_val, y_val = X.iloc[val_index, ], G.iloc[val_index, ], y[val_index]

    # Create gpb data
    train = gpb.Dataset(X_train, y_train)
    
    # Create random effects model
    gp_model = gpb.GPModel(
        group_data = G_train,
        likelihood = "gaussian",
        seed = random_state
    )

    # Create params dict
    params = {
        "random_state": random_state,
        "n_estimators": int(best_tunes.iloc[0, :]["user_attrs_n_rounds"]),
        "num_leaves": best_tunes.iloc[0, :]["params_num_leaves"],
        "min_child_samples": best_tunes.iloc[0, :]["params_min_child_samples"],
        "learning_rate": best_tunes.iloc[0, :]["params_learning_rate"],
        "min_child_weight": best_tunes.iloc[0, :]["params_min_child_weight"],
        "reg_alpha": best_tunes.iloc[0, :]["params_l1_reg"],
        "reg_lambda": best_tunes.iloc[0, :]["params_l2_reg"],
        "colsample_bytree": best_tunes.iloc[0, :]["params_colsample_bytree"]
    }
    
    
       # Train booster
    model = gpb.train(
        params = params,
        train_set = train,
        gp_model = gp_model
        )

    # Predict
    preds = model.predict(X_val, group_data_pred = G_val, predict_var = True, pred_latent = True)


The least populated class in y has only 1 members, which is less than n_splits=4.
Found `n_estimators` in params. Will use it instead of argument


[GPBoost] [Info] Total Bins 2665
[GPBoost] [Info] Number of data points in the train set: 1236, number of used features: 16
[GPBoost] [Info] [GPBoost with gaussian likelihood]: initscore=68.724595
[GPBoost] [Info] Start training from score 68.724595


In [27]:
pd.Series(preds["fixed_effect"]).describe()

count   413.000000
mean     68.982159
std       1.071233
min      64.241105
25%      68.283193
50%      68.890858
75%      69.636179
max      73.246134
dtype: float64

In [30]:
pd.Series(preds["random_effect_mean"]).describe()

count   413.000000
mean      1.382035
std       8.138955
min     -20.362094
25%      -3.593503
50%       3.629688
75%       6.568090
max      14.827909
dtype: float64

Both fixed & random effects make a considerably similar contribution to predictions.